In [ ]:
import os
import json
import yaml

# 클래스 ID 매핑 딕셔너리
class_map = {}
current_class_id = 0

def delete_image_file(image_path):
    try:
        if os.path.exists(image_path):
            os.remove(image_path)
            print(f"Deleted image file: {image_path}")
        else:
            print(f"Image file not found for deletion: {image_path}")
    except Exception as e:
        print(f"Error deleting image file {image_path}: {e}")

def process_json_to_txt(image_folder, json_folder, mode="train"):
    global class_map
    global current_class_id

    # 클래스 매핑을 저장할 경로 지정 (train 모드에서만 저장)
    if mode == "train":
        class_map_file = os.path.join(image_folder, f'{mode}_class_map.yaml')
    else:
        # val 모드에서는 기존 클래스 매핑 파일을 로드
        class_map_file = os.path.join(train_image_folder, 'train_class_map.yaml')
        if os.path.exists(class_map_file):
            with open(class_map_file, 'r', encoding='utf-8') as f:
                class_map = yaml.safe_load(f)
            print(f"Loaded class map from {class_map_file}")
        else:
            print(f"Error: Class map file not found at {class_map_file}")
            return

    print(f"Processing {mode} data...")
    print(f"Image folder: {image_folder}")
    print(f"JSON folder: {json_folder}")

    # 모든 하위 폴더를 재귀적으로 탐색
    for root, dirs, files in os.walk(json_folder):
        for json_file in files:
            if json_file.endswith('.json'):
                # JSON 파일 경로
                json_path = os.path.join(root, json_file)

                # 해당 JSON 파일에 대응하는 이미지 폴더 경로 계산
                relative_path = os.path.relpath(root, json_folder)
                image_subfolder = os.path.join(image_folder, relative_path)

                # 이미지 폴더가 존재하는지 확인
                if not os.path.isdir(image_subfolder):
                    print(f"Warning: Corresponding image folder not found for {json_path}")
                    continue

                # JSON 파일 읽기 (인코딩 지정)
                try:
                    with open(json_path, 'r', encoding='utf-8') as f:
                        data = json.load(f)
                except Exception as e:
                    print(f"Error reading JSON file {json_path}: {e}")
                    continue

                # SmallCategoryId 추출
                try:
                    small_category_id = data['rawDataInfo']['SmallCategoryId']
                except KeyError:
                    print(f"Warning: 'rawDataInfo' or 'SmallCategoryId' not found in {json_path}. Skipping and deleting image.")
                    # 이미지 파일 삭제
                    image_file_name = os.path.splitext(json_file)[0] + '.jpg'
                    image_path = os.path.join(image_subfolder, image_file_name)
                    delete_image_file(image_path)
                    continue

                # 클래스가 이미 매핑되어 있지 않으면 추가 (train 모드에서만)
                if mode == "train":
                    if small_category_id not in class_map:
                        class_map[small_category_id] = current_class_id
                        current_class_id += 1
                else:
                    if small_category_id not in class_map:
                        print(f"Warning: Class {small_category_id} not found in class map. Skipping and deleting image.")
                        # 이미지 파일 삭제
                        image_file_name = os.path.splitext(json_file)[0] + '.jpg'
                        image_path = os.path.join(image_subfolder, image_file_name)
                        delete_image_file(image_path)
                        continue

                # 클래스 ID 가져오기
                class_id = class_map[small_category_id]

                # 바운딩 박스 정보 추출
                try:
                    objects = data['learningDataInfo']['objects']
                except KeyError:
                    print(f"Warning: 'learningDataInfo' or 'objects' not found in {json_path}. Skipping and deleting image.")
                    # 이미지 파일 삭제
                    image_file_name = os.path.splitext(json_file)[0] + '.jpg'
                    image_path = os.path.join(image_subfolder, image_file_name)
                    delete_image_file(image_path)
                    continue

                # 이미지 파일명 설정
                try:
                    image_file_name = data['sourceDataInfo']['sourceDataID'] + '.jpg'
                    label_file_name = data['sourceDataInfo']['sourceDataID'] + '.txt'
                except KeyError:
                    print(f"Warning: 'sourceDataInfo' or 'sourceDataID' not found in {json_path}. Skipping and deleting image.")
                    # 이미지 파일 삭제
                    image_file_name = os.path.splitext(json_file)[0] + '.jpg'
                    image_path = os.path.join(image_subfolder, image_file_name)
                    delete_image_file(image_path)
                    continue

                # 이미지 경로 설정
                image_path = os.path.join(image_subfolder, image_file_name)
                label_path = os.path.join(image_subfolder, label_file_name)  # 이미지 폴더 내에 라벨 파일 저장

                # 이미지가 존재하는지 확인
                if not os.path.exists(image_path):
                    print(f"Warning: Image file not found at {image_path}")
                    continue

                # 이미지 크기 정보 (JSON에서 추출)
                try:
                    image_width, image_height = map(int, data['rawDataInfo']['resolution'].split('*'))
                except KeyError:
                    print(f"Warning: 'resolution' not found in {json_path}. Skipping and deleting image.")
                    delete_image_file(image_path)
                    continue

                # 라벨 파일 작성
                label_lines = []
                for obj in objects:
                    # 'bbox' 어노테이션만 처리
                    if obj.get('annotation') == 'bbox':
                        if 'coords' in obj:
                            coords = obj['coords']
                            x_min = coords['tl']['x']
                            y_min = coords['tl']['y']
                            x_max = coords['br']['x']
                            y_max = coords['br']['y']

                            # YOLO 형식으로 변환
                            x_center = (x_min + x_max) / 2 / image_width
                            y_center = (y_min + y_max) / 2 / image_height
                            bbox_width = (x_max - x_min) / image_width
                            bbox_height = (y_max - y_min) / image_height

                            # 라벨 작성
                            label_lines.append(f"{class_id} {x_center} {y_center} {bbox_width} {bbox_height}\n")
                        else:
                            print(f"Warning: 'coords' key not found in bbox object. Skipping object in file {json_path}")
                    else:
                        # 'bbox'가 아닌 어노테이션은 건너뜁니다.
                        continue

                # 라벨 파일 저장 (인코딩 지정)
                if label_lines:
                    with open(label_path, 'w', encoding='utf-8') as label_file:
                        label_file.writelines(label_lines)
                else:
                    print(f"Warning: No valid bbox objects found in {json_path}. Label file not created.")
                    # 이미지 파일 삭제
                    delete_image_file(image_path)

    # 클래스 매핑 파일 저장 (train 모드에서만)
    if mode == "train":
        with open(class_map_file, 'w', encoding='utf-8') as f:
            yaml.dump(class_map, f, allow_unicode=True)
        print(f"Class map YAML file saved at {class_map_file}")

# 사용 예시
if __name__ == '__main__':
    train_image_folder = r'H:/Data_AI_HUB/553/090.주행_차량_관점의_특수_차량_형상_데이터/01.데이터/1.Training/원천데이터_0906_add/'
    train_json_folder = r'H:/Data_AI_HUB/553/090.주행_차량_관점의_특수_차량_형상_데이터/01.데이터/1.Training/라벨링데이터_1007_add/'
    val_image_folder = r'H:/Data_AI_HUB/553/090.주행_차량_관점의_특수_차량_형상_데이터/01.데이터/2.Validation/원천데이터_0906_add/L_화물차/10_화물카고'
    val_json_folder = r'H:/Data_AI_HUB/553/090.주행_차량_관점의_특수_차량_형상_데이터/01.데이터/2.Validation/라벨링데이터_1007_add/L_화물차/10_화물카고'

    # 트레인 JSON 처리
    # process_json_to_txt(train_image_folder, train_json_folder, mode="train")

    # 밸리데이션 JSON 처리
    process_json_to_txt(val_image_folder, val_json_folder, mode="val")
